
# <p style="text-align: center;  margin-bottom: -1cm;">War Machine</p>
## <p style="text-align: center; margin-bottom: 0cm;">A predictive analysis of war utilizing graph theory and support vector machine models.</p>

#### <p style="text-align: center; margin-bottom: -1cm;">Lowell Monis, Kyle Cowden, Joseph Burke, Adhvik Kolar, Saif-ur-Rehman Sheikh</p>
#### <p style="text-align: center;">November 19, 2024</p>

***

This semester project has been completed in partial fulfillment of the requirements for Computational Mathematics, Sciences, and Engineering 202, Computational Modeling Tools and Techniques, taught by Dr. Nathan Haut during the Fall of 2024.

***

## Preamble

Before going through this project, one must ensure that they run the following cell to install modules and packages if they haven't been installed on the local server. The code is in a `raw` cell, so it must be changed to a `code` cell before it is run. This is a safety feature to avoid unnecessary installations if the notebook is reset and all cells are run.

***

This study will make use of the following Python modules, libraries, and packages. It is essential to run the following cell to avoid running into errors while viewing this notebook. The following cell also contains setup commands for styles that may be used in the notebook.

***

In [2]:
# Matplotlib Setup

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight') # Setting plot style

# Numpy Setup
import numpy as np

# Pandas Setup
import pandas as pd

# Seaborn Setup
import seaborn as sns
sns.set_context('paper')

# DateTime Setup
from datetime import timedelta, datetime, date

# Miscellaneous
import warnings # To hide any and all warnings and ensure for a cleaner notebook
warnings.filterwarnings('ignore')